In [53]:

import xgboost as xgb
import numpy as np
import matplotlib.pylab as plt
plt.style.use('ggplot')
#from feature_engineering import add_retning
from sklearn.model_selection import train_test_split, KFold
import pandas as pd
import sys
import lightgbm as lgb

sys.path.append('../data')

from display_data import import_data


HEr kommer funksjonene

In [54]:
def fix_missing_value(df):
    #layout, set everythin to 1
    df['layout'].fillna(int(1.0),inplace=True)



    #found correleation between number of stories and balconies,loggias
    df.loc[(df['stories'] < 7.0)&df['loggias'].isna(),'loggias'] = 0
    df.loc[(df['stories'] < 7.0)&df['balconies'].isna(),'balconies'] = 1

    df.loc[((df['stories'] > 12) & (df['stories'] < 16))|((df['stories'] > 17) & (df['stories'] < 24))&(df['loggias'].isna()),'loggias'] = 1
    df.loc[((df['stories'] > 12) & (df['stories'] < 16))|((df['stories'] > 17) & (df['stories'] < 24))&(df['loggias'].isna()),'balconies'] = 0


    #use the correleation between balconies and loggias to replace nans,
    df.loc[(df['loggias'] == 0.0 )& df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['balconies'] == 0.0 )& df['loggias'].isna(),'loggias'] = 1
    df.loc[(df['loggias'] > 0.0 )& df['balconies'].isna(),'balconies'] = 0
    df.loc[(df['balconies'] > 0.0 )& df['loggias'].isna(),'loggias'] = 0

    #use correlation between material and ceiling groups
    ceiling_under_2 = len(df[((df['material'] == 4)|(df['material'] == 3)) & df['ceiling'].isna()])
    sample = df[df['ceiling'] < 2.6]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[((df['material'] == 4)|(df['material'] == 3)) & (df['ceiling'].isna()),'ceiling'] = sample
    #use corelation between condition and ceiling groups
    ceiling_under_2 = len(df[(df['condition'] == 1) & df['ceiling'].isna()])
    sample = df[df['ceiling'] < 2.6]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[(df['condition'] == 1) & df['ceiling'].isna(),'ceiling'] = sample


    #use correleation betwwen ceiling and area
    ceiling_over_3 = len(df[(df['area_total'] > 104)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] > 3.0]['ceiling'].sample(n=ceiling_over_3,replace=True).to_numpy()
    df.loc[(df['area_total'] > 104)&(df['ceiling'].isna() ),'ceiling'] = sample

    ceiling_over_3 = len(df[(df['area_total'] > 75)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] > 2.95]['ceiling'].sample(n=ceiling_over_3,replace=True).to_numpy()
    df.loc[(df['area_total'] > 75)&(df['ceiling'].isna() ),'ceiling'] = sample

    ceiling_under_2 = len(df[(df['area_total'] < 55)&(df['ceiling'].isna())])
    sample = df[df['ceiling'] < 2.8]['ceiling'].sample(n=ceiling_under_2,replace=True).to_numpy()
    df.loc[(df['area_total'] < 55)&(df['ceiling'].isna() ),'ceiling'] = sample

    #seller
    #use correleation between seller and new
    df.loc[(df['new'] == 1 )& df['seller'].isna(),'seller'] = 3

    #use correleation between seller and constructed
    df.loc[(df['constructed'] > 2018)& df['seller'].isna(),'seller'] = 3
    df.loc[(df['constructed'] < 1990)& df['seller'].isna(),'seller'] = 1

    #use correleation between seller and stories
    df.loc[(df['stories'] > 17)& df['seller'].isna(),'seller'] = 3
    df.loc[(df['stories'] < 9)& df['seller'].isna(),'seller'] = 1

    #use correleation between seller and district
    df.loc[((df['district'] == 3)|(df['district']==6))& df['seller'].isna(),'seller'] = 1
    df.loc[((df['district'] == 11)|(df['district']==2))& df['seller'].isna(),'seller'] = 3

    #conditon
    #use correleation between condition and new
    df.loc[(df['new'] == 1)& df['condition'].isna(),'condition'] = 0

    #use correleation between condition and constructed year
    df.loc[(df['constructed'] >= 2017)& df['condition'].isna(),'condition'] = 0
    df.loc[(df['constructed'] <= 1989)& df['condition'].isna(),'condition'] = 1

    #material
    #use correlation between material and district
    df.loc[(df['district'] == 3)& df['material'].isna(),'material'] = 3

    #use correleation between material and constructed year
    df.loc[((df['constructed'] > 1970)& (df['constructed'] <= 1989))&df['material'].isna(),'material'] = 3
    df.loc[(df['constructed'] <= 1970)&df['material'].isna(),'material'] = 0

    #use correleation between stories and material
    df.loc[(df['stories'] < 7)& df['material'].isna(),'material'] = 0
    df.loc[((df['stories'] > 7)&(df['stories'] < 9))& df['material'].isna(),'material'] = 3

    #use correleation between conditon and material
    df.loc[(df['condition'] == 3)& df['material'].isna(),'material'] = 3
    df.loc[(df['condition'] == 0)& df['material'].isna(),'material'] = 2

    #use correleation between parking and material
    df.loc[((df['parking'] == 3)|(df['parking'] == 2))& df['material'].isna(),'material'] = 2

    #user correlation between area and material
    df.loc[(df['area_total'] > 104)& df['material'].isna(),'material'] = 2

    #parking
    #use coreletion between parking and material
    df.loc[((df['material'] == 0)|(df['material'] == 3))& df['parking'].isna(),'parking'] = 1

    #use correleation between stories and parking
    df.loc[((df['stories'] > 30))& df['parking'].isna(),'parking'] = 0
    df.loc[((df['stories'] > 7)&(df['stories'] < 9))& df['parking'].isna(),'parking'] = 1

    #use correleation between radius and parking
    df.loc[(df['radius'] < 0.0547)& df['parking'].isna(),'parking'] = 0
    df.loc[(df['radius'] > 0.251)& df['parking'].isna(),'parking'] = 1

    #windows_court and windows_street
    #windows_street and balconies
    df.loc[(df['balconies'] == 2)& df['windows_street'].isna(),'windows_street'] = 1

    #windows street and area
    df.loc[(df['area_total'] > 106)& df['windows_street'].isna(),'windows_street'] = 1
    df.loc[(df['area_total'] < 47)& df['windows_street'].isna(),'windows_street'] = 0

    #windwos_street and stories
    df.loc[(df['stories'] > 40)& df['windows_street'].isna(),'windows_street'] = 1
    df.loc[(df['stories'] < 9)& df['windows_street'].isna(),'windows_street'] = 0

    #windows_street and windows_court
    df.loc[(df['windows_street'] == 0)& df['windows_court'].isna(),'windows_court'] = 1

    df.loc[((df['windows_street'] == 1)& (df['area_total'] > 75)) & df['windows_court'].isna(),'windows_court'] = 1
    df.loc[((df['windows_street'] == 1)& (df['area_total'] < 47)) & df['windows_court'].isna(),'windows_court'] = 0

    #balconies
    #balconies and stories
    df.loc[((df['stories'] >= 12)& (df['area_total'] <= 16)) & df['balconies'].isna(),'balconies'] = 0
    df.loc[(df['stories'] < 7) & df['balconies'].isna(),'balconies'] = 1
    df.loc[((df['stories'] > 17)& (df['area_total'] <= 24)) & df['balconies'].isna(),'balconies'] = 0

    #matrial and balconies
    df.loc[(df['material'] == 0) & df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['material'] == 2) & df['balconies'].isna(),'balconies'] = 0

    #balconies and constructed
    df.loc[(df['constructed'] < 1970) & df['balconies'].isna(),'balconies'] = 1
    df.loc[(df['constructed'] > 2018) & df['balconies'].isna(),'balconies'] = 0

    #loggias
    #loggias and balconies
    df.loc[(df['balconies'] == 0) & df['loggias'].isna(),'loggias'] = 1
    df.loc[(df['balconies'] > 0) & df['loggias'].isna(),'loggias'] = 0

    #loggias and stories
    df.loc[(df['stories'] < 7) & df['loggias'].isna(),'loggias'] = 0
    df.loc[((df['stories'] >= 17)& (df['area_total'] <= 24)) & df['loggias'].isna(),'loggias'] = 1

    #bathrooms shared, bathroomes private

    #loggias and blaconies



    return df


def add_radius(df):
    #adds radius column to dataframe
    df['radius'] = np.sqrt((df['latitude']-55.75)**2 + (df['longitude']-37.55)**2)
    return df

def group_by_feature_and_price(df, feature, data_train = pd.DataFrame()):
    #makes a new column with the mean price for each group to all rows
    if data_train.empty:
        grouped = df.groupby([feature])

        mean_price = grouped['price'].mean()

        df_merged = pd.merge(df,mean_price, on=feature, how='left')

        column_name = str(feature + '_price')

        df[column_name] = df_merged['price_y']
    else:
        grouped = data_train.groupby([feature])

        mean_price = grouped['price'].mean()

        df_merged = pd.merge(df,mean_price, on=feature, how='left')

        column_name = str(feature + '_price')

        df[column_name] = df_merged['price']
    return df

def one_hot_encode(df, column_to_encode):
    """one hots encode for one singel column"""
    encoded_df = pd.get_dummies(df[[column_to_encode]].astype(str))
    df = pd.concat([df,encoded_df],axis=1)
    return df

def one_hot_encode_multiple(df, list_of_columns):
    """takes in multiple columns and runs one hot encode for each column"""
    for column_to_encode in list_of_columns:
        #print(column_to_encode)
        df = one_hot_encode(df, column_to_encode)
    return df

def add_direction(df):
    """adds direction to dataframe, can be one of eight categories (N,S,W,E)"""
    #straight up (north)
    normal_vector = np.array([0,1])
    #normal_vector = np.tile(normal_vector,(df.shape[0],1))
    #normal_vector = normal_vector.reshape((2,-df.shape[0]))
    temp = pd.DataFrame()
    temp['latitude'] = df['latitude']-55.75
    temp['longitude'] = df['longitude']-37.56


    apartment_vector = temp[['latitude','longitude']].to_numpy()


    #print(np.shape(apartment_vector), np.shape(normal_vector))
    angles = []
    for vector in apartment_vector:
        if vector[0] < 0:
            temp_angle = -angle_between(vector,normal_vector)
        else:
            temp_angle = angle_between(vector,normal_vector)
        angles.append(temp_angle)

    angles = [element * 10 for element in angles]


    angles_series = pd.Series(np.array(angles))
    #angles_series.plot.hist()

    df['direction'] = angles_series
    max = df.direction.max().round()
    min = df.direction.min().round()
    bins = [min,min*7/8,min*5/8,min*3/8,min/8,max/8,max*3/8,max*5/8,max*7/8,max]
    rounded_bins = [element.round() for element in bins]
    #print(rounded_bins)
    direction = pd.cut(df.direction, bins= rounded_bins,labels=['S','SW','W','NW','N','NE','E','SE','S'],ordered=False)
    df['direction'] = direction
    return df

def unit_vector(vector):
    """returns the unit vector if the vector"""
    return vector/np.linalg.norm(vector)

def angle_between(v1,v2):
    """returns angle between two vectors in radian"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u,v2_u),-1,1))

def fix_radius(data_test):
    """should just be called on test data"""
    #fix all radius issues
    data_test._set_value(23,'radius',0.203899)
    data_test._set_value(90,'radius',0.203899)
    data_test._set_value(2511,'radius',0.218159)
    data_test._set_value(5090,'radius',0.218159)
    data_test._set_value(6959,'radius',0.218159)
    data_test._set_value(8596,'radius',0.218159)
    data_test._set_value(4719, 'radius',0.19580)
    data_test._set_value(9547, 'radius',0.19520)
    data_test._set_value(2529, 'radius', np.sqrt((37.464994-37.55)**2+(55.627666-55.75)**2))

    return data_test

def drop_n_largest(data_train):
    """drops 4 largest values, should only be called on training set"""
    #drop 4 largest from training data, maybe not samrt, but we will see
    data_train.drop([3217,21414,15840,13938])
    return data_train

def clean_data(all_data):
    """cleans the data with all the knowledge we have so far"""


    #fix ceiling issues
    all_data.loc[all_data['ceiling'] > 50,'ceiling']*=0.01
    all_data.loc[all_data['ceiling'] > 25, 'ceiling']*=0.1
    all_data.loc[all_data['ceiling'] < 0.5,'ceiling'] = float('NaN')
    """"
    #fix area_kitchen and area_living issues
    all_data['living'] = all_data.area_living/all_data.area_total
    all_data['kitchen'] = all_data.area_total/all_data.area_kitchen

    all_data.loc[all_data['living'] > 1,'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['living'] > 1,'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100

    all_data.loc[all_data.area_living/all_data.area_total > 1, 'area_living'] = float('NaN')

    all_data['living'] = all_data.area_living/all_data.area_total
    all_data['kitchen'] = all_data.area_kitchen/all_data.area_total

    all_data['sum_area'] = all_data.area_living + all_data.area_kitchen
    all_data.loc[all_data['sum_area'] == 100, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 100, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100

    #this fixed some few rows.
    #when printing we see some other very clear "precentage situations, fixing these
    all_data.loc[all_data['sum_area'] == 38.5, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 38.5, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 52.7, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 52.7, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 71.6, 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[all_data['sum_area'] == 71.6, 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 20), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 20), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 15), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 15), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 10), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 10), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 30), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 80) & (all_data['area_kitchen'] == 30), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 10), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 10), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 15), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 15), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 20), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 20), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 25), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 25), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 30), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 30), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 50), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 90) & (all_data['area_kitchen'] == 50), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 60) & (all_data['area_total'] > 120), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] == 60) & (all_data['area_total'] > 120), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] < 70) & (all_data['area_total'] > 120)&(all_data['rooms'] < 3), 'area_living'] = all_data['area_living'] * all_data['area_total']/100
    all_data.loc[(all_data['sum_area'] < 70) & (all_data['area_total'] > 120)&(all_data['rooms'] < 3), 'area_kitchen'] = all_data['area_kitchen'] * all_data['area_total']/100
    """
    return all_data

def add_high_up(df):
    """add exponetital function to determine how high up a building is"""
    high_up = df.floor/df.stories
    high_up_exp = np.exp(high_up) - 1
    euler = np.exp(1)

    df['high_up'] = high_up_exp
    df['high_up'].where(df['high_up'] > euler, euler)


    return df

I want to use import_data function from another directory, and make the data ready for the model. LightGBM accepts categorical values, but they have to be encoded as no-negative integers.

In [55]:
#We dont want our model to care about the id of the house or the seller
data, data_test = import_data()
data_light, data_test_light = import_data()

Y = data.price
test_id = data_test.id

radius = True
# Add radius
if(radius):
    data = add_radius(data)
    data_test = add_radius(data_test)
    #coordinates = ['latitude', 'longitude']
    #data = data.drop(columns = coordinates)
    #data_test = data_test.drop(columns = coordinates)
    data_light = add_radius(data_light)
    data_test_light = add_radius(data_test_light)

fix_missing = False
if fix_missing:
    data = fix_missing_value(data)
    data_test = fix_missing_value(data_test)
    data_light = fix_missing_value(data_light)
    data_test_light = fix_missing_value(data_test_light)


categorical_data = ['seller','layout', 'windows_court', 'windows_street', 'condition', 'building_id','new','district','street',
                    'address', 'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating']

data_columns = list(data.columns)
numerical_data = list(set(data_columns)-set(categorical_data))
print(numerical_data)
for column in numerical_data:
    mean = data[column].mean()
    data[column] = data[column].replace(np.NaN, mean)
    if column != 'price':
        mean_test = data_test[column].mean()
        data_test[column] = data_test[column].replace(np.NAN,mean)

#Features
radius = True
district_mean_price = True
ohe = True
direction = True
cleaning = True
high_up = True
#add high up


if(high_up):
    data = add_high_up(data)
    data_test = add_high_up(data_test)
    data_light = add_high_up(data_light)
    data_test_light = add_high_up(data_test_light)


if(cleaning):
    data_test = fix_radius(data_test)
    data = clean_data(data)
    data_test = clean_data(data_test)
    data_test_light = fix_radius(data_test_light)
    data_light = clean_data(data_light)
    data_test_light = clean_data(data_test_light)


data = data.drop(columns = ['area_living', 'area_kitchen'])
data_test = data_test.drop(columns = ['area_living', 'area_kitchen'])


if(district_mean_price):
    data = group_by_feature_and_price(data,'district')
    data_test = group_by_feature_and_price(data_test,'district',data_train=data)
    data = data.drop(columns=['price', 'id'])


if(direction):
    data = add_direction(data)
    data_test = add_direction(data_test)
    data = one_hot_encode(data, 'direction')
    data_test = one_hot_encode(data_test, 'direction')
    data = data.drop(columns = 'direction')
    data_test = data_test.drop(columns = 'direction')

    data_light = add_direction(data_light)
    data_test_light = add_direction(data_test_light)

#One hot encode data
if(ohe):
    encode_categorical_data = ['seller','layout', 'windows_court', 'windows_street', 'condition','new','district',
                     'material', 'elevator_without', 'elevator_passenger', 'elevator_service', 'parking','garbage_chute', 'heating']
    data = one_hot_encode_multiple(data,encode_categorical_data)
    data_test = one_hot_encode_multiple(data_test,encode_categorical_data)



#Drop cat_data
data = data.drop(columns = categorical_data)
data_test = data_test.drop(columns=['id'])
data_test = data_test.drop(columns = categorical_data)
#print(data.info())

#chose which columns to foucs on light gbm
data_train_focus_light = data_light[['seller','material','district','condition','parking','new','direction','constructed','elevator_without','garbage_chute','loggias','layout','balconies','floor','high_up','rooms','area_total','radius','ceiling']]
data_test_focus_light = data_test_light[['seller','material','district','condition','parking','new','direction','constructed','elevator_without','garbage_chute','loggias','layout','balconies','floor','high_up','rooms','area_total','radius','ceiling']]
print(type(data_light))
print(data_light[0:2][['seller','material']])
print(data_train_focus_light[0:2])

X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.2, random_state=42)
X_train_light, X_test_light, y_train_light, y_test_light = train_test_split(data_train_focus_light, Y, test_size=0.2, random_state=42)

['bathrooms_private', 'loggias', 'id', 'phones', 'bathrooms_shared', 'area_kitchen', 'area_total', 'ceiling', 'radius', 'area_living', 'price', 'rooms', 'floor', 'constructed', 'balconies', 'latitude', 'longitude', 'stories']
<class 'pandas.core.frame.DataFrame'>
   seller  material
0     3.0       3.0
1     NaN       3.0
   seller  material  district  condition  parking  new direction  constructed  \
0     3.0       3.0      11.0        NaN      1.0  1.0        SW       2021.0   
1     NaN       3.0       2.0        3.0      1.0  0.0        NE       2010.0   

   elevator_without  garbage_chute  loggias  layout  balconies  floor  \
0               0.0            NaN      NaN     NaN        NaN    2.0   
1               0.0            NaN      0.0     1.0        1.0   18.0   

    high_up  rooms  area_total    radius  ceiling  
0  0.248849    2.0        59.2  0.218159     2.65  
1  1.054433    3.0        88.0  0.161215      NaN  


In [56]:
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    print("problem")
    assert (y_true >= 0).all()
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5

#data.info()
#print(len(Y))
#print(data.corr())
kaggle = False
if(kaggle):
    XGB_model = xgb.XGBRegressor(n_estimators = 3000,eta = 0.2,subsample=0.8)
    Y = np.log(Y)/np.log(15)
    XGB_model.fit(data,Y)
    result_XGB = XGB_model.predict(data_test)

    XGB_model = xgb.XGBRegressor(n_estimators = 5000,eta = 0.1,subsample=0.6)
    XGB_model.fit(data,Y)
    result_XGB_2 = XGB_model.predict(data_test)

    lightGBM_model = lgb.LGBMRegressor(
    categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12),
    num_leaves=9**2,
    max_depth=9,
    random_state=42,
    metric='rmsle',
    num_iterations=1000,
    learning_rate=0.09,
    bagging_freq = 5,
    bagging_fraction = 0.8)

    lightGBM_model.fit(data_train_focus_light,Y,categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12))
    result_light = lightGBM_model.predict(data_test_focus_light)

    lightGBM_model = lgb.LGBMRegressor(
    categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12),
    num_leaves=7**2,
    max_depth=7,
    random_state=50,
    metric='rmsle',
    num_iterations=4000,
    learning_rate=0.08,
    bagging_freq = 10,
    bagging_fraction = 0.6)

    lightGBM_model.fit(data_train_focus_light,Y,categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12))
    result_light_2 = lightGBM_model.predict(data_test_focus_light)


    prediction_kaggle = (result_light + result_XGB + result_light_2 + result_XGB_2)/4

else:   

    def pred_negative(dataset):
        for data in dataset:
            if(data<0):
                print(data)
                return True
        return False
    
    #deklarerer de ulike modellene med hyperparametre
    XGB_model_first = xgb.XGBRegressor(n_estimators = 1000,subsample=0.9)

    XGB_model_second = xgb.XGBRegressor(n_estimators = 2000,eta=0.1, subsample=0.8)

    lightGBM_model_first = lgb.LGBMRegressor(
    categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12),
    num_leaves=9**2,
    max_depth=9,
    random_state=42,
    metric='rmsle',
    num_iterations=1000,
    learning_rate=0.09,
    bagging_freq = 5,
    bagging_fraction = 0.8)

    lightGBM_model_second = lgb.LGBMRegressor(
    categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12),
    num_leaves=7**2,
    max_depth=7,
    random_state=42,
    metric='rmsle',
    num_iterations=2000,
    learning_rate=0.07,
    bagging_freq = 10,
    bagging_fraction = 0.7)

    #og legger til i en liste
    models=[XGB_model_first, XGB_model_second, lightGBM_model_first, lightGBM_model_second]

   
    #y_train = np.log(y_train)/np.log(15)

   
    n_splits=4
    kf = KFold(n_splits=n_splits, shuffle=False) #kan antageligvis ha på true
    x_training, x_holdout, y_training, y_holdout = train_test_split(data, Y, test_size=0.2, random_state=42)
    x_holdout_light, y_holdout_light = X_test_light, y_test_light

    

    x_splits = [next(kf.split(x_training)) for i in range(n_splits)]
    y_splits = [next(kf.split(y_training)) for i in range(n_splits)]
    #y_train_light = np.log(y_train_light)/np.log(15)
    

    y_pred=[]
    y_pred_holdout=[[]]

    x_train_lightGBM_first, x_test_lightGBM_first, y_train_lightGBM_first, y_test_lightGBM_first = X_train_light.iloc[x_splits[2][0]], X_train_light.iloc[x_splits[2][1]], y_train_light.iloc[x_splits[2][0]], y_train_light.iloc[x_splits[2][1]]
    #first lightGBM

    lightGBM_model_first.fit(x_train_lightGBM_first, y_train_lightGBM_first, categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12))
    y_pred = np.append(y_pred, lightGBM_model_first.predict(x_test_lightGBM_first))
    x_debug=x_test_lightGBM_first
    x_debug['y_pred'] = y_pred
    no_negative=0
    for index, row in x_debug.iterrows():
        
        if row['y_pred']<0:
            no_negative+=1

    print(no_negative)
            
    
    y_pred_holdout = np.append(y_pred_holdout, [lightGBM_model_first.predict(x_holdout_light)], axis=0)

    print("lightgbm: ", pred_negative(y_train_lightGBM_first))

    #first xgb
    #x_train_xgb_first, x_test_xgb_first, y_train_xgb_first, y_test_xgb_first = train_test_split(x_training.iloc[x_splits[0][0]], y_training.iloc[y_splits[0][0]], test_size=0.20, random_state=42)
    x_train_xgb_first, x_test_xgb_first, y_train_xgb_first, y_test_xgb_first = x_training.iloc[x_splits[0][0]], x_training.iloc[x_splits[0][1]], y_training.iloc[y_splits[0][0]], y_training.iloc[y_splits[0][1]]
    XGB_model_first.fit(x_train_xgb_first, y_train_xgb_first)
    y_pred = np.append(y_pred, XGB_model_first.predict(x_test_xgb_first))
    y_pred_holdout=XGB_model_first.predict(x_holdout)
    y_pred_holdout = [XGB_model_first.predict(x_holdout)]


    #second xgb
    x_train_xgb_second, x_test_xgb_second, y_train_xgb_second, y_test_xgb_second = x_training.iloc[x_splits[1][0]], x_training.iloc[x_splits[1][1]], y_training.iloc[y_splits[1][0]], y_training.iloc[y_splits[1][1]]    #print(len(x_train_xgb_first[0]), len(y_train_xgb_first[0]))
    XGB_model_second.fit(x_train_xgb_second, y_train_xgb_second)
    y_pred = np.append(y_pred, XGB_model_second.predict(x_test_xgb_second))
    y_pred_holdout = np.append(y_pred_holdout, [XGB_model_second.predict(x_holdout)], axis=0)

    categories_lightGBM = ['seller','material','district','condition','parking','new','direction','constructed','elevator_without','garbage_chute','loggias','layout','balconies','floor','high_up','rooms','area_total','radius','ceiling']
    #lightgbm
    


    x_train_lightGBM_second, x_test_lightGBM_second, y_train_lightGBM_second, y_test_lightGBM_second = X_train_light.iloc[x_splits[3][0]], X_train_light.iloc[x_splits[3][1]], y_train_light.iloc[y_splits[3][0]], y_train_light.iloc[y_splits[3][1]]

    lightGBM_model_second.fit(x_train_lightGBM_second, y_train_lightGBM_second, categorical_feature=(0,1,2,3,4,5,6,7,8,9,10,11,12))
    y_pred = np.append(y_pred, lightGBM_model_second.predict(x_test_lightGBM_second))
    y_pred_holdout = np.append(y_pred_holdout, [lightGBM_model_second.predict(x_holdout_light)], axis=0)

    print("lightgbm2: ", pred_negative(y_pred))


    #averaging the holdout prediction
    average_holdout=[]
    for i in range(y_pred_holdout.shape[1]):
        temp_average=(y_pred_holdout[0,i]+y_pred_holdout[1,i]+y_pred_holdout[2,i]+y_pred_holdout[3,i])/y_pred_holdout.shape[0]
        average_holdout = np.append(average_holdout, temp_average)

    #metalearner
    XGB_model_meta = xgb.XGBRegressor(n_estimators = 1000,subsample=0.9)
    x_training['y_pred']=y_pred
    x_holdout['holdout_pred'] = average_holdout
    XGB_model_meta.fit(x_training, y_training)
    final_prediction = XGB_model_meta.predict(x_holdout)
    result = root_mean_squared_log_error(y_holdout, final_prediction)
    print(result)
    
        

[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


c:\Users\gunna\tdt4173-task-2\moscow_housing\venv\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\gunna\tdt4173-task-2\moscow_housing\venv\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
c:\Users\gunna\tdt4173-task-2\moscow_housing\venv\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
c:\Users\gunna\tdt4173-task-2\moscow_housing\venv\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict i

4


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 4657

In [ ]:
    train = pd.DataFrame()
    print(prediction.shape, y_train.shape)
    train['one'] =prediction
    train['two'] = result_light
    train['three'] =prediction_2
    train['four'] =result_light_2

    from sklearn import linear_model
    glm = linear_model.LogisticRegression( random_state=1, solver='lbfgs', max_iter=200, fit_intercept=True, penalty='none', verbose=0)
    glm.fit(train, y_test)
    prediction = glm.predict(train)

    #prediction = (prediction + result_light + prediction_2  + result_light_2 )/4

##############



NameError: name 'prediction' is not defined

In [ ]:



#lgb.plot_importance(lightGBM_model)
#print(data.test)

if(kaggle):
    prediction_kaggle = 15**prediction_kaggle
    submission = pd.DataFrame()
    submission['id'] = test_id
    submission['price_prediction'] = prediction_kaggle
    submission.to_csv('submission_XGB_EDA.csv', index= False)
else:
    prediction = 15**prediction
    rmsle = root_mean_squared_log_error(y_test,prediction)
    print("first run", rmsle)


Runs
Only numeric, no feature engineered: 0.19744699138034857

Radius added: 0.20967165360778273, Why worse

Distric_mean: 0.19084295853893113

Both Radius and district: 0.196656529295351

One Hot encoding and district: 0.18454859183640202

One HOt, district, loggg: 0.14149042970609157

one_hot, district, log, radius: 0.14492800769942604

one_hot, district, log, radius, long, lat: 0.14029707272730435

one_hot, district, log, radius, long, lat, 10 000 iter:0.1366021699810636

blblbla med cleaning, 1000 iter: 0.14343726755323022

uten clenaing 1000 iter: 0.13981074583345976

blblba med semi- cleaning, 1000 iter: 0.1389066060558281

blbla med semi-cleaning, uten area kitch og den andre, 1000 iter: 0.13715263327959132  gir 0.17 på kaggle

stacking med fire varainter: 0.127, stacking med seks varianter ga samme, vil anta at de ikke var ulike nok.
